In [1]:
%pip install pip -U -q
%pip install -U boto3 python-dotenv -q

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [3]:
!  echo "AWS_S3_BUCKET=${AWS_S3_BUCKET:-models}" > .env
!  echo "AWS_S3_ENDPOINT=${AWS_S3_ENDPOINT:-http://minio.minio.svc:9000}" >> .env
!  echo "AWS_ACCESS_KEY_ID=$(oc -n minio extract secret/minio-root-user --keys=MINIO_ROOT_USER --to=-)" >> .env
!  echo "AWS_SECRET_ACCESS_KEY=$(oc -n minio extract secret/minio-root-user --keys=MINIO_ROOT_PASSWORD --to=-)" >> .env

# MINIO_ROOT_USER
# MINIO_ROOT_PASSWORD


In [4]:
import os, boto3
from dotenv import load_dotenv

load_dotenv()

True

Lets use the NVIDIA Triton model folder structure to store the saved models

Triton model folder structure:

models (provide this dir as source / MODEL_REPOSITORY )
└─ [ model name ]
    └─ 1 (version)
        └── model.savedmodel
            ├── saved_model.pb

In [5]:
local_path = "../models"
remote_path = "gpt2"

bucket = os.getenv("AWS_S3_BUCKET", "models")

s3 = boto3.client(
    "s3",
    endpoint_url=os.getenv("AWS_S3_ENDPOINT", "http://minio.minio.svc:9000"),
    aws_access_key_id=os.getenv("AWS_ACCESS_KEY_ID", "minioadmin"),
    aws_secret_access_key=os.getenv("AWS_SECRET_ACCESS_KEY", "minioadmin"),
)


if bucket not in [bu["Name"] for bu in s3.list_buckets()["Buckets"]]:
    s3.create_bucket(Bucket=bucket)


def uploadDirectory(path, bucketname):
    for root, dirs, files in os.walk(path):
        for file in files:
            print(f"uploading: {file} to {bucket}/{remote_path}")
            s3.upload_file(
                os.path.join(root, file), bucketname, f"{remote_path}/{file}"
            )
            print("[ok]")


uploadDirectory(path=local_path, bucketname=bucket)

# with open(f'{local_path}/{filename}', "rb") as f:
#     s3.upload_fileobj(f, bucket, f'gpt2/{filename}')
# print(f'uploaded: {bucket}/gpt2/{filename}')

uploading: gpt2_lm_v1.keras to models/gpt2
[ok]
uploading: .gitkeep to models/gpt2
[ok]
